# data pre-processing
* not using genre

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

# prepare data
---

In [ ]:
ls -la ../input

In [ ]:
dir_data_src = '../input/anime-recommendations-database'
os.listdir(dir_data_src)

## load anime.csv

In [ ]:
anime_csv = pd.read_csv(os.path.join(dir_data_src, 'anime.csv'))
print(anime_csv.shape)
anime_csv.index = anime_csv.anime_id.values
anime_csv.head()

In [ ]:
np.sort(anime_csv.anime_id.values)

In [ ]:
anime_csv_dic = {}
for ii in range(anime_csv.shape[0]):
    anime_id = anime_csv.anime_id.values[ii]
    anime_name = anime_csv.name.values[ii]
    anime_csv_dic[anime_id] = anime_name

anime_csv_dic

## load rating.csv

In [ ]:
'''
load rating.csv
'''
rating_csv = pd.read_csv(os.path.join(dir_data_src, 'rating.csv'))
print(rating_csv.shape)
rating_csv.head()

In [ ]:
np.unique(rating_csv.anime_id.values)

In [ ]:
'''
anime not exist in rating.csv
'''
anime_csv.iloc[np.isin(anime_csv.anime_id.values, np.unique(rating_csv.anime_id.values), invert=True),:]

In [ ]:
'''
`Yuri!!! on Ice` doesn't exist in rating.csv
'''
rating_csv[rating_csv.anime_id=='32995']

In [ ]:
'''Natsume Yuujinchou Go'''
rating_csv[rating_csv.anime_id=='32983']

In [ ]:
np.unique(rating_csv.user_id.values), len(np.unique(rating_csv.user_id.values))

In [ ]:
np.unique(rating_csv.rating.values)

In [ ]:
rating_csv.rating.values != -1

## create rating_csv2
* omit rating == -1

In [ ]:
rating_csv2 = rating_csv[rating_csv.rating.values != -1].copy()
print(rating_csv2.shape)
rating_csv2.head(10)

## Anime no-rated

In [ ]:
rating_csv2.groupby('anime_id').size().keys().values

In [ ]:
anime_csv.anime_id.values

In [ ]:
np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)

In [ ]:
'''
anime_id no rated
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]

In [ ]:
'''
how much
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)].shape

In [ ]:
'''
show anime no rated
'''
tmp = anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
print(anime_csv.loc[tmp].shape)
anime_csv.loc[tmp]

In [ ]:
rating_csv2.query('anime_id in @tmp')

## create anime_csv2
* Excluding anime_id not in rating_csv2

In [ ]:
12294 - 2368

In [ ]:
tmp = anime_csv.anime_id.values[np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
anime_csv2 = anime_csv.loc[tmp].copy()
print(anime_csv2.shape)
anime_csv2

# dic_anime
* use anime_csv2

In [ ]:
#['anime_id-'+str(ee) for ee in anime_csv.anime_id.values]
dic_anime = gensim.corpora.Dictionary([['anime_id-'+str(ee) for ee in anime_csv2.anime_id.values]], prune_at=None)
print(len(dic_anime))
dic_anime

In [ ]:
dic_anime.doc2bow(['anime_id-1', 'anime_id-1', 'anime_id-5', 'anime_id-6'])

In [ ]:
dic_anime[0], dic_anime.doc2idx(['anime_id-1', 'anime_id-6'])

# user_dic

In [ ]:
user_dic = {}
for ee in np.unique(rating_csv.user_id.values):
    user_dic[ee] = 'user_id-' + str(ee)

len(user_dic)

In [ ]:
np.unique(rating_csv2.user_id.values)

In [ ]:
dic_user = gensim.corpora.Dictionary([['PH_DUMMY']], prune_at=None)
dic_user.add_documents([['user_id-' + str(ee) for ee in np.unique(rating_csv2.user_id.values)]])
#dic_user.add_documents([['genre-'+ee for ee in genre]])
len(user_dic), len(dic_user)

In [ ]:
dic_user[0], dic_user[1], dic_user[2]

# create csr matrix

In [ ]:
rating_csv2

In [ ]:
rating_csv2.anime_id

In [ ]:
rating_csv2[rating_csv2.anime_id==11617]

In [ ]:
%%time
rating_csv2[rating_csv2.anime_id==int(re.sub('^anime_id-', '', dic_anime[0]))]

In [ ]:
from collections.abc import Sequence

class Seq(Sequence):
    
    def __init__(self, anime_csv=anime_csv2, rating_csv=rating_csv2, dic_anime=dic_anime, dic_user=dic_user):
        self.anime_csv = anime_csv
        self.rating_csv = rating_csv
        self.dic_anime = dic_anime # doc
        self.dic_user = dic_user # word
        
        print(self.anime_csv.shape, len(self.dic_anime))
        assert self.anime_csv.shape[0] == len(self.dic_anime)
    
    def __getitem__(self, idx):
        return self._get(idx)
    
    del_anime_id = re.compile('^anime_id-')
    def _get(self, idx):
        try:
            anime_id = int(self.del_anime_id.sub('', self.dic_anime[idx]))
            #print(anime_id)
        except KeyError:
            raise IndexError
        df = self.rating_csv[rating_csv2.anime_id==anime_id]
        users = [self.dic_user.token2id['user_id-'+str(ee)] for ee in df.user_id.values]
        return list(zip(*(users, df.rating.values)))
    
    def __len__(self):
        return len(self.dic_anime)

In [ ]:
seq = Seq()

In [ ]:
%%time
seq[0]

In [ ]:
(ee for ee in seq)

In [ ]:
%%time

seq = Seq()
corpus_csr = gensim.matutils.corpus2csc(
    (ee for ee in seq),
    num_terms=max(dic_user.keys())+1
).T
corpus_csr.shape

In [ ]:
list(zip(*seq[0]))[0]
corpus_csr[0].todense()[:,list(zip(*seq[0]))[0]][:,:20]

In [ ]:
%%time
scipy.sparse.save_npz('corpus_csr', corpus_csr)
dic_user.save('word_dic')
dic_anime.save('doc_dic')

# Check if a model can be created

In [ ]:
from feature_eng import WordAndDoc2vec, calc_gsim

In [ ]:
%%time
wd2v = WordAndDoc2vec(corpus_csr, dic_user, dic_anime)
wd2v

In [ ]:
num_features = 100
wd2v.make_model(num_features=num_features, embeddings_val=0.1)

In [ ]:
wd2v.max_num_prod, wd2v.num_neg